In [1]:
#importing required libraries 
import import_ipynb
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

from scipy import stats

from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD

from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

# Dataset

In [2]:
m_df = pd.read_csv('data/movies_metadata.csv')
m_df.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,To

In [3]:
m_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
m_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

# Preprocessing

# Remove columns

In [5]:
m_df = m_df.drop('adult', axis=1)
m_df = m_df.drop('homepage',axis=1)
m_df = m_df.drop('original_title', axis=1)
m_df = m_df.drop('poster_path', axis=1)
m_df = m_df.drop('production_companies', axis=1)
m_df = m_df.drop('production_countries', axis=1)
m_df = m_df.drop('runtime', axis=1)

In [6]:
m_df.shape

(45466, 17)

In [7]:
m_df.drop_duplicates(subset=['title'], inplace=True)
m_df.shape

(42278, 17)

# cleaning dataset

In [8]:
m_df['genres'] = m_df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] 
                                                                       if isinstance(x, list) else [])

In [9]:
m_df

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45459,NaN,0,[Science Fiction],222848,tt0112613,en,It's the year 3000 AD. The world's most danger...,0.661558,1995-01-01,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Caged Heat 3000,False,3.5,1.0
45461,NaN,0,"[Drama, Family]",439050,tt6209470,fa,Rising and falling between a man and woman.,0.072051,NaN,0.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,NaN,0,[Drama],111109,tt2028550,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45464,NaN,0,[],227506,tt0008536,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


# Popularity based Recommender

In [10]:
# Claculation of c 
vote_counts = m_df[m_df['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = m_df[m_df['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.2515079121035075

In [11]:
## Claculation of m
m = vote_counts.quantile(0.95)
m

430.0

In [12]:
# extracting release year from release_date
m_df['year'] = pd.to_datetime(m_df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] 
                                                                           if x != np.nan else np.nan)

In [13]:
col_list = ['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']
qualified = m_df[(m_df['vote_count'] >= m) 
                 & (m_df['vote_count'].notnull()) 
                 & (m_df['vote_average'].notnull())][col_list]

# converting vote_count and vote_average colums to integer
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.head(10)

,title,year,vote_count,vote_average,popularity,genres
0,Toy Story,1995,5415,7,21.946943,"[Animation, Comedy, Family]"
1,Jumanji,1995,2413,6,17.015539,"[Adventure, Fantasy, Family]"
5,Heat,1995,1886,7,17.924927,"[Action, Crime, Drama, Thriller]"
9,GoldenEye,1995,1194,6,14.686036,"[Adventure, Action, Thriller]"
15,Casino,1995,1343,7,10.137389,"[Drama, Crime]"
17,Four Rooms,1995,539,6,9.026586,"[Crime, Comedy]"
18,Ace Ventura: When Nature Calls,1995,1128,6,8.205448,"[Crime, Comedy, Adventure]"
31,Twelve Monkeys,1995,2470,7,12.297305,"[Science Fiction, Thriller, Mystery]"
33,Babe,1995,756,6,14.404764,"[Fantasy, Drama, Comedy, Family]"
38,Clueless,1995,828,6,9.882383,"[Comedy, Drama, Romance]"


In [14]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [15]:
qualified['weighted_rating'] = qualified.apply(weighted_rating, axis=1)

In [16]:
qualified = qualified.sort_values('weighted_rating', ascending=False).head(250)

In [17]:
qualified.head(10)

,title,year,vote_count,vote_average,popularity,genres,weighted_rating
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.918521
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.906933
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.898265
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.883078
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.873219
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.870126
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.865515
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.863464
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.862207
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.853568


# Based on genre

In [18]:
#cleaning genres column
temp = m_df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
temp.name = 'genre'
mgen_df = m_df.drop('genres', axis=1).join(temp)

In [19]:
def make_toplist(genre, percentile=0.85):
    df = mgen_df[mgen_df['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    col_list = ['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genre']
    qualified = df[(df['vote_count'] >= m) 
                   & (df['vote_count'].notnull()) 
                   & (df['vote_average'].notnull())][col_list]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['weighted_rating'] = qualified.apply(lambda x: 
                                                   (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), 
                                                   axis=1)
    qualified = qualified.sort_values(by=['weighted_rating'], ascending=False).head(10)
    #final = ((qualified_top250['title']).to_string(index=False))
    return qualified

In [20]:
x=make_toplist('Action').head(10)
df = pd.DataFrame(x['title'])
df

,title
15480,Inception
12481,The Dark Knight
4863,The Lord of the Rings: The Fellowship of the Ring
7000,The Lord of the Rings: The Return of the King
5814,The Lord of the Rings: The Two Towers
256,Star Wars
1154,The Empire Strikes Back
4135,Scarface
9430,Oldboy
1910,Seven Samurai


# ContentBasedRecommendation

In [37]:
small_mdf = pd.read_csv('data/links_small.csv')
small_mdf = small_mdf[small_mdf['tmdbId'].notnull()]['tmdbId'].astype('int')
small_mdf

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int32

In [38]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [39]:
m_df['id'] = m_df['id'].apply(convert_int)

In [40]:
m_df['overview']

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45459    It's the year 3000 AD. The world's most danger...
45461          Rising and falling between a man and woman.
45462    An artist struggles to finish his work while a...
45464    In a small town live two brothers, one a minis...
45465    50 years after decriminalisation of homosexual...
Name: overview, Length: 42278, dtype: object

In [41]:
sm_df = m_df[m_df['id'].isin(small_mdf)]
sm_df.shape

(8695, 18)

In [42]:
sm_df.drop_duplicates(subset=['title'], inplace=True)

In [43]:
sm_df.shape

(8695, 18)

# Description based Recommendation

In [44]:
sm_df['tagline'] = sm_df['tagline'].fillna('')
sm_df['description'] = sm_df['overview'] + sm_df['tagline']
sm_df['description'] = sm_df['description'].fillna('')

In [45]:
sm_df

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,description
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ..."
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,A family wedding reignites the ancient feud be...
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom..."
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39784,NaN,8000000,[Drama],159550,tt0255313,en,A man must cope with the loss of his wife and ...,0.038998,2001-09-23,0.0,[],Released,,The Last Brickmaker in America,False,7.0,1.0,2001,A man must cope with the loss of his wife and ...
40004,NaN,1000000,"[Thriller, Romance]",392572,tt5165344,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,2016-08-12,0.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,2016,"Rustom Pavri, an honourable officer of the Ind..."
40058,NaN,15050000,"[Adventure, Drama, History, Romance]",402672,tt3859980,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,2016-08-11,16180000.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,,Mohenjo Daro,False,6.7,26.0,2016,"Village lad Sarman is drawn to big, bad Mohenj..."
40224,NaN,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",315011,tt4262980,ja,From the mind behind Evangelion comes a hit la...,9.285519,2016-07-29,77000000.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,2016,From the mind behind Evangelion comes a hit la...


In [46]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(sm_df['description'])
tfidf_matrix.shape

(8695, 258647)

In [47]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [48]:
cosine_sim.shape

(8695, 8695)

In [49]:
sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])

In [50]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [51]:
get_recommendations('The Dark Knight').head(10)

7647                      The Dark Knight Rises
132                              Batman Forever
1110                             Batman Returns
7920    Batman: The Dark Knight Returns, Part 2
7314                 Batman: Under the Red Hood
524                                      Batman
7619                           Batman: Year One
2547               Batman: Mask of the Phantasm
2664                                        JFK
7865    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

# Metadata based Recommendation

In [54]:
credits = pd.read_csv('data/credits.csv')
keywords = pd.read_csv('data/keywords.csv')

In [55]:
credits.head(10)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [56]:
keywords.head(4)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."


In [57]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [58]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
m_df['id'] = m_df['id'].astype('int')
m_df.shape

(42278, 18)

In [59]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int32 
 1   keywords  46419 non-null  object
dtypes: int32(1), object(1)
memory usage: 544.1+ KB


In [60]:
# merging require details with our main dataframe 
m_df = m_df.merge(credits, on='id')
m_df = m_df.merge(keywords, on='id')

In [61]:
m_df

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43237,NaN,0,[Science Fiction],222848,tt0112613,en,It's the year 3000 AD. The world's most danger...,0.661558,1995-01-01,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Caged Heat 3000,False,3.5,1.0,1995,"[{'cast_id': 1, 'character': 'Kira (as Cassand...","[{'credit_id': '5757f36ac3a3687d6f000e8a', 'de...",[]
43238,NaN,0,"[Drama, Family]",439050,tt6209470,fa,Rising and falling between a man and woman.,0.072051,NaN,0.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...","[{'id': 10703, 'name': 'tragic love'}]"
43239,NaN,0,[Drama],111109,tt2028550,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,2011,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
43240,NaN,0,[],227506,tt0008536,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",[]


In [62]:
sm_df = m_df[m_df['id'].isin(small_mdf)]
sm_df.shape

(8759, 21)

In [63]:
sm_df.drop_duplicates(subset=['title'], inplace=True)

In [64]:
sm_df['cast'] = sm_df['cast'].apply(literal_eval)
sm_df['crew'] = sm_df['crew'].apply(literal_eval)
sm_df['keywords'] = sm_df['keywords'].apply(literal_eval)
sm_df['cast_size'] = sm_df['cast'].apply(lambda x: len(x))
sm_df['crew_size'] = sm_df['crew'].apply(lambda x: len(x))

In [65]:
sm_df

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",26,16
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",7,4
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",10,10
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38061,NaN,8000000,[Drama],159550,tt0255313,en,A man must cope with the loss of his wife and ...,0.038998,2001-09-23,0.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,2001,"[{'cast_id': 1, 'character': 'Henry Cobb', 'cr...","[{'credit_id': '544475aac3a36819fb000578', 'de...","[{'id': 6054, 'name': 'friendship'}, {'id': 20...",7,2
38264,NaN,1000000,"[Thriller, Romance]",392572,tt5165344,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,2016-08-12,0.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,2016,"[{'cast_id': 0, 'character': 'Rustom Pavri', '...","[{'credit_id': '5951baf692514129c4016600', 'de...","[{'id': 10540, 'name': 'bollywood'}]",14,16
38313,NaN,15050000,"[Adventure, Drama, History, Romance]",402672,tt3859980,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,2016-08-11,16180000.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Mohenjo Daro,False,6.7,26.0,2016,"[{'cast_id': 0, 'character': 'Sarman', 'credit...","[{'credit_id': '57cd5d3592514179d50018e8', 'de...","[{'id': 10540, 'name': 'bollywood'}]",12,16
38472,NaN,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",315011,tt4262980,ja,From the mind behind Evangelion comes a hit la...,9.285519,2016-07-29,77000000.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. 

In [66]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [67]:
sm_df['director'] = sm_df['crew'].apply(get_director)

In [68]:
sm_df['cast'] = sm_df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
sm_df['cast']
sm_df['cast'] = sm_df['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [69]:
sm_df.head(10)

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106,John Lasseter
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",26,16,Joe Johnston
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",7,4,Howard Deutch
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",10,10,Forest Whitaker
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",12,7,Charles Shyer
5,NaN,60000000,"[Action, Crime, Drama, Thriller]",949,tt0113277,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,1995-12-15,187436818.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995,"[Al Pacino, Robert De Niro, Val Kilmer]","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '...",65,71,Michael Mann
6,NaN,58000000,"[Comedy, Romance]",11860,tt0114319,en,An ugly duckling having undergone a remarkable...,6.677277,1995-12-15,0.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995,"[Harrison Ford, Julia Ormond, Greg Kinnear]","[{'credit_id': '52fe44959251416c75039da9', 'de...","[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam...",57,53,Sydney Pollack
7,NaN,0,"[Action, Adventure, Drama, Family]",45325,tt0112302,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995,"[Jonathan Taylor Thomas, Brad Renfro, Rachael ...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[],7,4,Peter Hewitt
8,NaN,35000000,"[Action, Adventure, Thriller]",9091,tt0114576,en,International action superstar Jean Claude Van...,5.23158,1995-12-22,64350171.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror 

In [70]:
sm_df['keywords'] = sm_df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [71]:
sm_df.head(10)

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,John Lasseter
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,Joe Johnston
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",7,4,Howard Deutch
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,Forest Whitaker
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",12,7,Charles Shyer
5,NaN,60000000,"[Action, Crime, Drama, Thriller]",949,tt0113277,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,1995-12-15,187436818.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995,"[Al Pacino, Robert De Niro, Val Kilmer]","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[robbery, detective, bank, obsession, chase, s...",65,71,Michael Mann
6,NaN,58000000,"[Comedy, Romance]",11860,tt0114319,en,An ugly duckling having undergone a remarkable...,6.677277,1995-12-15,0.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995,"[Harrison Ford, Julia Ormond, Greg Kinnear]","[{'credit_id': '52fe44959251416c75039da9', 'de...","[paris, brother brother relationship, chauffeu...",57,53,Sydney Pollack
7,NaN,0,"[Action, Adventure, Drama, Family]",45325,tt0112302,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995,"[Jonathan Taylor Thomas, Brad Renfro, Rachael ...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[],7,4,Peter Hewitt
8,NaN,35000000,"[Action, Adventure, Thriller]",9091,tt0114576,en,International action superstar Jean Claude Van...,5.23158,1995-12-22,64350171.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror 

In [72]:
sm_df['cast'] = sm_df['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [73]:
sm_df.head(10)

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,John Lasseter
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,Joe Johnston
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",7,4,Howard Deutch
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,Forest Whitaker
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",12,7,Charles Shyer
5,NaN,60000000,"[Action, Crime, Drama, Thriller]",949,tt0113277,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,1995-12-15,187436818.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995,"[alpacino, robertdeniro, valkilmer]","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[robbery, detective, bank, obsession, chase, s...",65,71,Michael Mann
6,NaN,58000000,"[Comedy, Romance]",11860,tt0114319,en,An ugly duckling having undergone a remarkable...,6.677277,1995-12-15,0.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995,"[harrisonford, juliaormond, gregkinnear]","[{'credit_id': '52fe44959251416c75039da9', 'de...","[paris, brother brother relationship, chauffeu...",57,53,Sydney Pollack
7,NaN,0,"[Action, Adventure, Drama, Family]",45325,tt0112302,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995,"[jonathantaylorthomas, bradrenfro, rachaelleig...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[],7,4,Peter Hewitt
8,NaN,35000000,"[Action, Adventure, Thriller]",9091,tt0114576,en,International action superstar Jean Claude Van...,5.23158,1995-12-22,64350171.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,S

In [74]:
sm_df['director'] = sm_df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
sm_df['director'] = sm_df['director'].apply(lambda x: [x,x])

In [75]:
sm_df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'imdb_id',
       'original_language', 'overview', 'popularity', 'release_date',
       'revenue', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'year', 'cast', 'crew', 'keywords',
       'cast_size', 'crew_size', 'director'],
      dtype='object')

# Keywords

In [76]:
s = sm_df.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [77]:
s = s.value_counts()
s[:5]

independent film        596
woman director          526
murder                  370
duringcreditsstinger    307
based on novel          292
Name: keyword, dtype: int64

In [78]:
s = s[s > 1]

In [79]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [80]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [81]:
sm_df['keywords'] = sm_df['keywords'].apply(filter_keywords)
sm_df['keywords'] = sm_df['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
sm_df['keywords'] = sm_df['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [82]:
sm_df['soup'] = sm_df['keywords'] + sm_df['cast'] + sm_df['director'] + sm_df['genres']
sm_df['soup'] = sm_df['soup'].apply(lambda x: ' '.join(x))

In [83]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(sm_df['soup'])

In [84]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [85]:
sm_df

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,soup
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...",13,106,"[johnlasseter, johnlasseter]",jealousi toy boy friendship friend rivalri boy...
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgam, disappear, basedonchildren'sbook, n...",26,16,"[joejohnston, joejohnston]",boardgam disappear basedonchildren'sbook newho...
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fish, bestfriend, duringcreditssting]",7,4,"[howarddeutch, howarddeutch]",fish bestfriend duringcreditssting waltermatth...
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",10,10,"[forestwhitaker, forestwhitaker]",basedonnovel interracialrelationship singlemot...
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[babi, midlifecrisi, confid, age, daughter, mo...",12,7,"[charlesshyer, charlesshyer]",babi midlifecrisi confid age daughter motherda...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38061,NaN,8000000,[Drama],159550,tt0255313,en,A man must cope with the loss of his wife and ...,0.038998,2001-09-23,0.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,2001,"[sidneypoitier, wendycrewson, jayo.sanders]","[{'credit_id': '544475aac3a36819fb000578', 'de...",[friendship],7,2,"[greggchampion, greggchampion]",friendship sidneypoitier wendycrewson jayo.san...
38264,NaN,1000000,"[Thriller, Romance]",392572,tt5165344,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,2016-08-12,0.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,2016,"[akshaykumar, ileanad'cruz, eshagupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],14,16,"[tinusureshdesai, tinusureshdesai]",bollywood akshaykumar ileanad'cruz eshagupta t...
38313,NaN,15050000,"[Adventure, Drama, History, Romance]",402672,tt3859980,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,2016-08-11,16180000.0,"[{'iso_639_1': 'hi', 'name

In [86]:
sm_df.drop_duplicates(subset=['title'], inplace=True)

In [87]:
sm_df

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,release_date,revenue,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,soup
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...",13,106,"[johnlasseter, johnlasseter]",jealousi toy boy friendship friend rivalri boy...
1,NaN,65000000,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgam, disappear, basedonchildren'sbook, n...",26,16,"[joejohnston, joejohnston]",boardgam disappear basedonchildren'sbook newho...
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fish, bestfriend, duringcreditssting]",7,4,"[howarddeutch, howarddeutch]",fish bestfriend duringcreditssting waltermatth...
3,NaN,16000000,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",10,10,"[forestwhitaker, forestwhitaker]",basedonnovel interracialrelationship singlemot...
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[babi, midlifecrisi, confid, age, daughter, mo...",12,7,"[charlesshyer, charlesshyer]",babi midlifecrisi confid age daughter motherda...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38061,NaN,8000000,[Drama],159550,tt0255313,en,A man must cope with the loss of his wife and ...,0.038998,2001-09-23,0.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,2001,"[sidneypoitier, wendycrewson, jayo.sanders]","[{'credit_id': '544475aac3a36819fb000578', 'de...",[friendship],7,2,"[greggchampion, greggchampion]",friendship sidneypoitier wendycrewson jayo.san...
38264,NaN,1000000,"[Thriller, Romance]",392572,tt5165344,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,2016-08-12,0.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,2016,"[akshaykumar, ileanad'cruz, eshagupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],14,16,"[tinusureshdesai, tinusureshdesai]",bollywood akshaykumar ileanad'cruz eshagupta t...
38313,NaN,15050000,"[Adventure, Drama, History, Romance]",402672,tt3859980,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,2016-08-11,16180000.0,"[{'iso_639_1': 'hi', 'name

In [88]:
sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])

In [89]:
get_recommendations('The Dark Knight').head(10)

7647                 The Dark Knight Rises
5984                         Batman Begins
7314            Batman: Under the Red Hood
6356                          The Prestige
1110                        Batman Returns
8457               Kidnapping Mr. Heineken
5735                              Thursday
1236                        Batman & Robin
2045                             Following
8540    Batman v Superman: Dawn of Justice
Name: title, dtype: object

# Combined model

In [90]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = sm_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [91]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7305,Inception,14075,8,2010,7.918521
6356,The Prestige,4510,8,2006,7.760759
7647,The Dark Knight Rises,9263,7,2012,6.922434
5984,Batman Begins,7511,7,2005,6.905320
7245,Kick-Ass,4747,7,2010,6.854771
524,Batman,2145,7,1989,6.708019
1008,M,465,8,1931,6.679495
7314,Batman: Under the Red Hood,459,7,2010,6.154273
2045,Following,363,7,1998,6.051890
8046,Kick-Ass 2,2275,6,2013,5.881016


# Collaborative filtering

In [92]:
reader = Reader()

In [93]:
ratings = pd.read_csv('data/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [94]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [95]:
svd = SVD()
svd

In [96]:
ratings[['userId', 'movieId', 'rating']]

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [97]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5)

{'test_rmse': array([0.89456697, 0.90615809, 0.89978922, 0.89451387, 0.89042469]),
 'test_mae': array([0.68835342, 0.69858559, 0.69337215, 0.68860213, 0.68538877]),
 'fit_time': (10.502469301223755,
  10.587647676467896,
  10.52994704246521,
  10.370397090911865,
  10.574392557144165),
 'test_time': (0.4157097339630127,
  0.28563857078552246,
  0.2521333694458008,
  0.2593262195587158,
  0.271242618560791)}

In [98]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [99]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [100]:
svd.predict(1, 3671, 3)

Prediction(uid=1, iid=3671, r_ui=3, est=3.0847173810160786, details={'was_impossible': False})

# Hybrid Recommendation model

In [101]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [102]:
id_map = pd.read_csv('data/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(sm_df[['title', 'id']], on='id').set_index('title')

In [103]:
id_map

,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0
...,...,...
The Last Brickmaker in America,161944,159550.0
Rustom,162542,392572.0
Mohenjo Daro,162672,402672.0


In [104]:
indices_map = id_map.set_index('id')

In [105]:
print(indices_map)

          movieId
id               
862.0           1
8844.0          2
15602.0         3
31357.0         4
11862.0         5
...           ...
159550.0   161944
392572.0   162542
402672.0   162672
315011.0   163056
391698.0   163949

[8695 rows x 1 columns]


In [106]:
indices.to_csv('indices1.csv')

In [108]:
sm_df.to_csv('tempdb.csv')

In [109]:
indices1 = pd.read_csv('indices1.csv')
indices1

,title,0
0,Toy Story,0
1,Jumanji,1
2,Grumpier Old Men,2
3,Waiting to Exhale,3
4,Father of the Bride Part II,4
...,...,...
8690,The Last Brickmaker in America,8690
8691,Rustom,8691
8692,Mohenjo Daro,8692
8693,Shin Godzilla,8693


In [110]:
print('The Dark Knight' in indices1.values)

True


In [111]:
def hybrid(userId, title):
    if title not in indices1.values:
        return "Movie not found"
    idx = indices[title]
    #print(title)
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    #print(movie_id)
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    #print(movie_indices)
    movies = sm_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [112]:
hybrid(1000, 'Sabrin')

'Movie not found'

In [113]:
hybrid(1, 'The Enforcer')

,title,vote_count,vote_average,year,id,est
3737,Dirty Harry,630.0,7.4,1971,984,2.990506
5671,Police Story,128.0,7.3,1985,9056,2.976130
5735,Thursday,84.0,7.0,1998,9812,2.948475
2398,Nighthawks,87.0,6.4,1981,21610,2.904166
6837,RockNRolla,851.0,6.9,2008,13809,2.871947
4044,Every Which Way But Loose,126.0,6.2,1978,15417,2.859474
3765,Spy Game,592.0,6.8,2001,1535,2.842072
693,Police Story 3: Supercop,109.0,6.7,1992,11134,2.831556
2587,El Mariachi,243.0,6.6,1992,9367,2.757612
7410,Nightfall,14.0,6.8,1957,34667,2.702677


In [117]:
hybrid(1, 'Nighthawks')

,title,vote_count,vote_average,year,id,est
5671,Police Story,128.0,7.3,1985,9056,2.976130
5735,Thursday,84.0,7.0,1998,9812,2.948475
6837,RockNRolla,851.0,6.9,2008,13809,2.871947
3765,Spy Game,592.0,6.8,2001,1535,2.842072
693,Police Story 3: Supercop,109.0,6.7,1992,11134,2.831556
8200,The Raid 2,832.0,7.6,2014,180299,2.789790
2587,El Mariachi,243.0,6.6,1992,9367,2.757612
4861,To Live and Die in L.A.,129.0,6.8,1985,9846,2.741172
7410,Nightfall,14.0,6.8,1957,34667,2.702677
8084,Machete Kills,642.0,5.3,2013,106747,2.690257


In [116]:
hybrid(1, 'Thursday')

,title,vote_count,vote_average,year,id,est
2508,Boiling Point,26.0,4.6,1993,22213,2.920392
2398,Nighthawks,87.0,6.4,1981,21610,2.904166
4737,Time and Tide,26.0,5.9,2000,49291,2.901879
6837,RockNRolla,851.0,6.9,2008,13809,2.871947
8285,Revenge of the Green Dragons,34.0,5.2,2014,286709,2.864260
8623,İtirazım Var,9.0,7.1,2014,265351,2.793834
2587,El Mariachi,243.0,6.6,1992,9367,2.757612
5729,Pusher,162.0,6.8,1996,2061,2.757173
7539,Takers,399.0,6.1,2010,22907,2.735617
217,The Glass Shield,9.0,5.4,1994,72031,2.710671


In [115]:
hybrid(1, 'Police Story')

,title,vote_count,vote_average,year,id,est
5062,Enter the Dragon,489.0,7.3,1973,9461,3.020851
5735,Thursday,84.0,7.0,1998,9812,2.948475
2398,Nighthawks,87.0,6.4,1981,21610,2.904166
6837,RockNRolla,851.0,6.9,2008,13809,2.871947
2276,Armour of God II: Operation Condor,109.0,7.0,1991,10975,2.845395
3647,The Big Brawl,27.0,5.9,1980,19267,2.843279
693,Police Story 3: Supercop,109.0,6.7,1992,11134,2.831556
4908,Crime Story,39.0,6.7,1993,18857,2.787241
2277,Armour of God,111.0,6.8,1986,10974,2.781469
6047,New Police Story,176.0,7.0,2004,11636,2.772746


In [114]:
hybrid(1, 'Dirty Harry')

,title,vote_count,vote_average,year,id,est
5,Heat,1886.0,7.7,1995,949,3.211900
3442,Something Wild,68.0,6.5,1986,11300,2.961197
3607,Sea of Love,144.0,6.7,1989,12150,2.950673
2398,Nighthawks,87.0,6.4,1981,21610,2.904166
5271,Escape from Alcatraz,581.0,7.3,1979,10734,2.844963
693,Police Story 3: Supercop,109.0,6.7,1992,11134,2.831556
2304,The Limey,110.0,6.6,1999,10388,2.828531
3500,The Dead Pool,227.0,6.3,1988,10651,2.815575
8200,The Raid 2,832.0,7.6,2014,180299,2.789790
4106,Blood Work,210.0,6.1,2002,9573,2.745857
